# 2: Regular Expressions, Text Normalization, Edit Distance

**2.1**: Write regular expressions for the following languages.


1.   the set of all alphabetic strings;
2.   the set of all lower case alphabetic strings ending in a *b*
3.   the set of all strings from the alphabet *a, b* such that each *a* is immediately preceded by and immediately followed by a *b*;



In [27]:
import re

strings = ["a_b0c", "Db", "AbcdE", "bbabbbabbb", "abab", "123", "1b", "bb"]

# We interpret "string" as an entire word in a list of words
print("1. The set of all alphabetic strings:")
for s in strings:
  match_1 = re.search(r'^[a-zA-Z]+$', s)
  if match_1:
    print(match_1.group())
print('\n')

print("2. The set of all lower-case alphabetic strings ending in a 'b':")
for s in strings:
  match_2 = re.search(r'^[a-z]*b$', s)
  if match_2:
    print(match_2.group())
print('\n')

print("3. The set of all strings from the alphabet [a,b] such that\n",\
      "each a is immediately preceded by and immediately followed by a b:")
for s in strings:
  match_3 = re.search(r'^b+(ab+)*b*$', s)
  if match_3:
    print(match_3.group())

1. The set of all alphabetic strings:
Db
AbcdE
bbabbbabbb
abab
bb


2. The set of all lower-case alphabetic strings ending in a 'b':
bbabbbabbb
abab
bb


3. The set of all strings from the alphabet [a,b] such that
 each a is immediately preceded by and immediately followed by a b:
bbabbbabbb
bb


**2.2**: Write regular expressions for the following languages. By "word", we mean an alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, and so forth.


1.   the set of all strings with two consecutive repeated words (e.g., "Humbert Humbert" and "the the" but not "the bug" or "the big bug");
2.   all strings that start at the beginning of the line with an integer and that end at the end of the line with a word;
3.   all strings that have both the word *grotto* and the word *raven* in them (but not, e.g., words llike *grottos* that merely *contain* the word *grotto*);
4.   write a pattern that places the first word of an English sentence in a register. Deal with punctuation.



In [ ]:
import re
